# Harry Potter sentence classifier

## Data preparation

### Load books from files

In [1]:
filenames = {
    1: "Book 1 - The Philosopher's Stone.txt",
    2: 'Book 2 - The Chamber of Secrets.txt',
    3: 'Book 3 - The Prisoner of Azkaban.txt',
    4: 'Book 4 - The Goblet of Fire.txt',
    5: 'Book 5 - The Order of the Phoenix.txt',
    6: 'Book 6 - The Half Blood Prince.txt',
    7: 'Book 7 - The Deathly Hallows.txt'
}

In [4]:
books = {}
for id, filename in filenames.items():
    with open('books/' + filename, 'r') as file:
        book = file.read().replace('\n', '')
    books[id] = book

In [ ]:
### Remove g

In [11]:
import re
footers = {
    1: r'Page \| \d Harry Potter and the Philosophers Stone - J.K. Rowling',
    2: r'Page \| \d Harry Potter and the Chamber of Secrets - J.K. Rowling',
    3: r'Page \| \d Harry Potter and the Prisoner of Azkaban - J.K. Rowling',
    4: r'Page \| \d Harry Potter and the Goblet of Fire - J.K. Rowling',
    5: r'Page \| \d Harry Potter and the Order of the Phoenix - J.K. Rowling',
    6: r'Page \| \d Harry Potter and the Half Blood Prince - J.K. Rowling',
    7: r'Page \| \d Harry Potter and the Deathly Hallows - J.K. Rowling'
}

In [18]:
for book, content in books.items():
    books[book] = re.sub(footers[book], '', content)

In [21]:
books[4][:5000]

'/ THE RIDDLE HOUSE The villagers of Little Hangleton still called it “the Riddle House,” even though it had been many years since the Riddle family had lived there. It stood on a hill overlooking the village, some of its windows boarded, tiles missing from its roof, and ivy spreading unchecked over its face. Once a fine-looking manor, and easily the largest and grandest building for miles around, the Riddle House was now damp, derelict, and unoccupied. The Little Hangletons all agreed that the old house was “creepy.” Half a century ago, something strange and horrible had happened there, something that the older inhabitants of the village still liked to discuss when topics for gossip were scarce. The story had been picked over so many times, and had been embroidered in so many places, that nobody was quite sure what the truth was anymore. Every version of the tale, however, started in the same place: Fifty years before, at daybreak on a fine summer’s morning, when the Riddle House had 